# Import libraries and define functions + Initial setup

In [1]:
## Import Libraries
import pandas as pd
import numpy as np
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from wordcloud import WordCloud
import csv


## Display all rows of pandas dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
'''
name: relative_transcript_abundance

purpose: calculate relative transcript abundance

input: a dataframe with a ref_gene_id column identifying the transcript gene of origin and a cov columns with 
the coverage for the transcripts.

output: the same dataframe with a relative abundance column added
'''



def relative_transcript_abundance(df):
    
    ## Group by gene_id and get total expression for each gene (not counting introns)
    df_sums = df[["gene_id", "total_CPM"]].groupby("gene_id").sum()
    df_sums["total_CPM_gene"] = df_sums["total_CPM"]
    df_sums.drop(columns="total_CPM", inplace=True)
    
    ## Merge dataframe with total gene level CPM with regular transcript level CPM dataframe
    merged_df = pd.merge(df, df_sums, how='inner', on="gene_id")
    
    ## Calculater relative percent abundance for each transcript within its gene
    merged_df["relative_abundance_percent"] = ((merged_df["total_CPM"]/merged_df["total_CPM_gene"]) * 100)
    
    ## Rename total_CPM for transcript column
    merged_df["total_CPM_transcript"] = merged_df["total_CPM"]
    merged_df.drop(columns="total_CPM", inplace=True)


    return merged_df

In [3]:
'''
function name: fix_column_names

purpose: Fixing the column names, making them smaller, informative, and consistent

input: The raw counts dataframe for either genes or transcripts 

output: Same dataframe with improved column names
'''

def fix_column_names(df, is_gene=False):
    
    ## Check if this is a gene counts object
    if is_gene:
        
        ## Get count column names and create list of new column names
        count_columns = df.columns.tolist()
        list_new_names = ["gene_id"]
        
        ## gene_id comes in as index for gene counts data, make it into the first column instead
        df["gene_id"] = df.index
        cols = list(df.columns)
        cols = [cols[-1]] + cols[:-1]
        df = df[cols]
        df.reset_index(inplace=True, drop=True)
    
    ## If it is a transcript dataset
    else:
        ## Set count columns and create list of new names
        count_columns = df.columns[2:].tolist()
        list_new_names = [ "transcript_id", "gene_id"]
    
    ## Fix names one by one and add to list of new names
    for col in count_columns:
        col = col.split("_mapped")[0] + "_counts"
        list_new_names.append(col)
    
    ## Rename columns
    df.columns = list_new_names
    
    return df 

In [4]:
'''
function name: parse_df_columns

purpose: parsing the last aggregate column of the gtf/gff3 into useful columns and cleaning non-relevant columns

input: dataframe containining "raw" gtf/gff

output: dataframe containing gtf with useful columns ["gene_id", "transcript_id", etc...]
'''

def parse_df_columns(df, is_ref=True, is_transcript=False, is_prot=False, delete_other=True):

    if is_ref:

        ## Get gene ids
        df["gene_id"] = df["other"].str.split('";', expand=True)[0].str.extract("([^ \"]*$)", expand=True)
        
        ## Get gene names
        df["gene_name"] = df["other"].str.split("gene_name \"", expand=True)[1].str.split('\";', expand=True)[0]
        
        ## Get get transcript biotype
        df["gene_biotype"] = df["other"].str.split('gene_biotype "', expand=True)[1].str.split('"', expand=True)[0]
        
        ## If is transcript get transcript id and transcript biotype
        if is_transcript:
            df["transcript_id"] = df["other"].str.split('transcript_id "', expand=True)[1].str.split('"', expand=True)[0]
            df["transcript_biotype"] = df["other"].str.split('transcript_biotype "', expand=True)[1].str.split('"', expand=True)[0]
            
            ## If is prot get protein_id
            if is_prot:
                df["protein_id"] = df["other"].str.split('protein_id "', expand=True)[1].str.split('"', expand=True)[0]
                df["ccds_id"] = df["other"].str.split('ccds_id "', expand=True)[1].str.split('"', expand=True)[0]
                df["exon_number"] = df["other"].str.split('exon_number "', expand=True)[1].str.split('"', expand=True)[0]

        ## Drop "other" column
        df.drop(columns=["other", "dot_1", "dot_2"], inplace=True)
        

    else:

        ## Get gene ids
        df["gene_id"] = df["other"].str.split('";', expand=True)[0].str.extract("([^ \"]*$)", expand=True)

        ## Get transcript ids
        df["transcript_id"] = df["other"].str.split('transcript_id "', expand=True)[1].str.split('"', expand=True)[0]

        ## Get exon number
        df["exon_number"] = df["other"].str.split('exon_number "', expand=True)[1].str.split('"', expand=True)[0]

        ## Drop "other" column
        if delete_other:
            df.drop(columns=["other", "dot_1", "dot_2"], inplace=True)

    for col in df.columns:
        df.loc[df[col].isnull(), col] = np.NaN
        

    return df

In [5]:
'''
function name: calculate_cpm

purpose: Calculate CPM for the each sample given

input: Counts dataset

output: Counts dataset with CPM columns as well
'''

def calculate_cpm(df, is_gene=False):

    ## Set count columns if dataframe is gene counts
    if is_gene:
        count_columns = df.columns[1:].tolist()
    
    ## Set count columns if dataframe is transcript counts
    else:
        count_columns = df.columns[2:].tolist()

    ## Loop through counts columns to calculate CPM and add to the dataframe
    for col in count_columns:
        
        df[col] = round(df[col], 2)
        cpm_name = col.replace("_counts", "_CPM")
        df[cpm_name] = round(((df[col]/(df[col].sum())) * 1000000), 2)
    
    return df  

In [6]:
## Open original reference
original_ref = pd.read_csv("../../references/Homo_sapiens.GRCh38.107_ERCC.gtf", header=None, delimiter="\t", low_memory=False, 
                       names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"], comment="#")

original_ref = original_ref.loc[~original_ref["chr"].str.startswith("ERCC-")]

## Parse through reference to get gene names and ids
orig_ref = original_ref.loc[original_ref["type"]=="gene"].copy()
orig_ref = parse_df_columns(orig_ref, is_ref=True)

In [7]:
glinos_comp = pd.read_csv("./glinos/ours_all_vs_glinos.tracking", sep="\t", header=None, usecols=[2,3,4],
                         names=["glinos", "category_glinos", "ours"])

In [8]:
leung_comp = pd.read_csv("./leung/our_all_vs_leung.tracking", sep="\t", header=None, usecols=[2,3,4],
                         names=["leung", "category_leung", "ours"])

In [9]:
df = leung_comp.merge(glinos_comp, on="ours", how="inner")

In [11]:
df.loc[df["category_glinos"] == "=", "is_in_glinos"] = True
df.loc[df["category_glinos"] != "=", "is_in_glinos"] = False

In [12]:
df.loc[df["category_leung"] == "=", "is_in_leung"] = True
df.loc[df["category_leung"] != "=", "is_in_leung"] = False

In [13]:
df.loc[((df["is_in_glinos"] == True) & (df["is_in_leung"] == True)), "is_in_both"] = True
df.loc[~((df["is_in_glinos"] == True) & (df["is_in_leung"] == True)), "is_in_both"] = False

In [14]:
df.loc[((df["is_in_glinos"] == True) | (df["is_in_leung"] == True)), "is_in_either"] = True
df.loc[~((df["is_in_glinos"] == True) | (df["is_in_leung"] == True)), "is_in_either"] = False

In [15]:
df["gene_id"] = df["ours"].str.split(":", expand=True)[1].str.split("|", expand=True)[0]
df["transcript_id"] = df["ours"].str.split(":", expand=True)[1].str.split("|", expand=True)[1]

In [16]:
df = df.merge(orig_ref[["gene_id", "gene_name", "chr"]], on="gene_id", how="left")

In [17]:
df_new_from_new = df.loc[df["gene_id"].str.contains("BambuGene")].copy()

df_new_from_known = df.loc[~df["gene_id"].str.contains("BambuGene")].copy() 

In [18]:
df_new_from_known_nuclear = df_new_from_known.loc[df_new_from_known["chr"] != "MT"].copy()
df_new_from_known_mito = df_new_from_known.loc[df_new_from_known["chr"] == "MT"].copy()

In [19]:
print("Number of new from known genes nuclear genes:", df_new_from_known_nuclear.shape[0])
print("Not exclusive to ours:", df_new_from_known_nuclear.loc[df_new_from_known_nuclear["is_in_either"] == True].shape[0])
print("Exclusive to ours:", df_new_from_known_nuclear.loc[df_new_from_known_nuclear["is_in_either"] == False].shape[0])
print("Found in Leung:", df_new_from_known_nuclear.loc[df_new_from_known_nuclear["is_in_leung"] == True].shape[0])
print("Found in Glinos:", df_new_from_known_nuclear.loc[df_new_from_known_nuclear["is_in_glinos"] == True].shape[0])
print("Found in Both:", df_new_from_known_nuclear.loc[df_new_from_known_nuclear["is_in_both"] == True].shape[0])

Number of new from known genes nuclear genes: 1534
Not exclusive to ours: 220
Exclusive to ours: 1314
Found in Leung: 141
Found in Glinos: 124
Found in Both: 45


In [20]:
print("Number of new isoforms from known mito genes:", df_new_from_known_mito.shape[0])
print("Not exclusive to ours:", df_new_from_known_mito.loc[df_new_from_known_mito["is_in_either"] == True].shape[0])
print("Exclusive to ours:", df_new_from_known_mito.loc[df_new_from_known_mito["is_in_either"] == False].shape[0])
print("Found in Leung:", df_new_from_known_mito.loc[df_new_from_known_mito["is_in_leung"] == True].shape[0])
print("Found in Glinos:", df_new_from_known_mito.loc[df_new_from_known_mito["is_in_glinos"] == True].shape[0])
print("Found in Both:", df_new_from_known_mito.loc[df_new_from_known_mito["is_in_both"] == True].shape[0])

Number of new isoforms from known mito genes: 34
Not exclusive to ours: 0
Exclusive to ours: 34
Found in Leung: 0
Found in Glinos: 0
Found in Both: 0


In [21]:
print("Number of new from known genes nuclear genes:", df_new_from_new.shape[0])
print("Not exclusive to ours:", df_new_from_new.loc[df_new_from_new["is_in_either"] == True].shape[0])
print("Exclusive to ours:", df_new_from_new.loc[df_new_from_new["is_in_either"] == False].shape[0])
print("Found in Leung:", df_new_from_new.loc[df_new_from_new["is_in_leung"] == True].shape[0])
print("Found in Glinos:", df_new_from_new.loc[df_new_from_new["is_in_glinos"] == True].shape[0])
print("Found in Both:", df_new_from_new.loc[df_new_from_new["is_in_both"] == True].shape[0])

Number of new from known genes nuclear genes: 1861
Not exclusive to ours: 9
Exclusive to ours: 1852
Found in Leung: 9
Found in Glinos: 0
Found in Both: 0


In [22]:
df_output = df[['gene_id', 'gene_name', 'transcript_id', 'chr', 'leung', 'glinos', 'is_in_glinos', 'is_in_leung', 'is_in_both', 'is_in_either']].copy()

In [23]:
df_output.to_csv("../../data/processed/compare_annotations/ours_vs_glinos_vs_leung_ALL_comparison.csv",
                index=False)